In [4]:
import numpy as np
import scipy.optimize as opti
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('whitegrid')
sns.set_context('poster')
from scipy.interpolate import griddata

In [5]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [6]:
import sys
sys.path.append('/Users/vonderlinden2/rsx_analysis/writing_to_vtk/source')
import structured_3d_vtk as struc_3d

In [108]:
spatial_increment = 0.001
x_min, x_max = -0.028, 0.025
y_min, y_max = -0.043, 0.039
z_min, z_max = 0.249, 0.249

bounds = ((x_min, x_max), (y_min, y_max), (z_min, z_max)) 

x_coord = np.linspace(x_min, x_max, np.ceil((x_max-x_min)/spatial_increment))
y_coord = np.linspace(y_min, y_max, np.ceil((y_max-y_min)/spatial_increment))

mesh = np.meshgrid(x_coord, y_coord, indexing='ij')
grid_points = np.dstack(map(np.ravel, mesh))[0]

In [129]:
file_path = '../output/2016-07-03/B_z0249'
ending = '.npz'

bx_measurements = struc_3d.read_idl('bx')
by_measurements = struc_3d.read_idl('by')
bz_measurements = struc_3d.read_idl('bz')

direction_measurements = [bx_measurements, by_measurements, bz_measurements] 


interpolated_vectors = []
for time_point in xrange(21):
    print time_point
    points = []
    values = []
    for measurements in direction_measurements:
        (points_direction,
         values_direction) = struc_3d.read_points_from_measurement_dict(measurements,
                                                                        time_point,
                                                                        [0.249])
        points.append(points_direction)
        values.append(values_direction)
    
    points[0] = np.delete(points[0], 2, axis=1)
    points[1] = np.delete(points[1], 2, axis=1)
    points[2] = np.delete(points[2], 2, axis=1)
    interpolated_vector = struc_3d.interpolate_vector(grid_points, points, values)
    interpolated_vector = struc_3d.add_vacuum_field(interpolated_vector)
    
    assert np.sum(np.isnan(interpolated_vector[0])) == 0
    assert np.sum(np.isnan(interpolated_vector[1])) == 0
    assert np.sum(np.isnan(interpolated_vector[2])) == 0
    
    interpolated_vectors.append(interpolated_vector)
    
    
    
    np.savez(file_path + str(time_point).zfill(4) + ending,
             b_x=interpolated_vector[0], 
             b_y=interpolated_vector[1],
             b_z=interpolated_vector[2])

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20


In [127]:
np.savez?